In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# OSeMOSYS Global Capacity Expansion Results for Nepal

## Read in OSeMOSYS Results

In [ ]:
annual_emissions = pd.read_csv('./model/Results/No_Emission_Limit/AnnualEmissions.csv')
capital_investments = pd.read_csv('./model/Results/No_Emission_Limit/CapitalInvestment.csv')
new_capacity = pd.read_csv('./model/Results/No_Emission_Limit/NewCapacity.csv')
demand = pd.read_csv('./model/Results/No_Emission_Limit/Demand.csv')
production_annual = pd.read_csv('./model/Results/No_Emission_Limit/ProductionByTechnologyAnnual.csv')
total_capacity = pd.read_csv('./model/Results/No_Emission_Limit/TotalCapacityAnnual.csv')

## Filter and Aggregate Results

In [ ]:
capital_investments = capital_investments.loc[capital_investments['TECHNOLOGY'].isin(['HYDRO', 'GAS', 'COAL'])].reset_index(drop=True)
new_capacity = new_capacity.loc[new_capacity['TECHNOLOGY'].isin(['HYDRO', 'GAS', 'COAL'])].reset_index(drop=True)
production_annual = production_annual.loc[production_annual['TECHNOLOGY'].isin(['HYDRO', 'GAS', 'COAL'])].reset_index(drop=True)
total_capacity = total_capacity.loc[total_capacity['TECHNOLOGY'].isin(['HYDRO', 'GAS', 'COAL'])].reset_index(drop=True)
demand = demand.groupby(by=['YEAR','FUEL','REGION']).sum().reset_index()

## Plot Results

### Installed Capacity 

In [ ]:
total_capacity_2 = total_capacity.pivot(index='YEAR',columns='TECHNOLOGY',values='VALUE')

ax = total_capacity_2.plot.bar( 
    stacked=True,
    legend='reverse',
    color=['tab:purple','tab:orange','tab:blue']
)

ax.set_ylabel('CAPACITY (GW)')
ax.set_xlabel('')
plt.locator_params(axis='x', nbins=11)
plt.xticks(rotation = -45)

for label in ax.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False)

### Electricity Generation

In [ ]:
def add_legend_line(legend):
    ax = legend.axes

    handles, labels = ax.get_legend_handles_labels()
    handles.append(mlines.Line2D([],[],color='k'))
    labels.append("Demand")
    for h,l in enumerate(labels):
        if l == 'VALUE':
            labels.remove(l)
            del handles[h]

    legend._legend_box = None
    legend._init_legend_box(handles, labels)
    legend._set_loc(legend._loc)
    legend.set_title(legend.get_title().get_text())

In [ ]:
production_annual_2 = production_annual.pivot(index='YEAR',columns='TECHNOLOGY',values='VALUE')

ax = production_annual_2.plot.bar( 
    stacked=True,
    legend='reverse',
    color=['tab:orange','tab:blue']
)

xticks = ax.get_xticks()

demand['VALUE'].plot(
    x=xticks,
    ax=ax,
    color='k')

ax.set_ylabel('GENERATION (PJ)')
ax.set_xlabel('')
plt.locator_params(axis='x', nbins=11)
plt.xticks(rotation = -45)

for label in ax.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False) 

legend = ax.legend()
add_legend_line(legend)

### Capital Investments 

In [ ]:
capital_investments_2 = capital_investments.pivot(index='YEAR',columns='TECHNOLOGY',values='VALUE')

ax = capital_investments_2.plot.bar( 
    stacked=True,
    legend='reverse',
    color=['tab:orange','tab:blue']
)

ax.set_ylabel('INVESTMENTS (M$)')
ax.set_xlabel('')
plt.locator_params(axis='x', nbins=11)
plt.xticks(rotation = -45)

for label in ax.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False) 

### Emissions

In [ ]:
ax = sns.barplot(
    x='YEAR', 
    y='VALUE', 
    hue='EMISSION', 
    data=annual_emissions
)

ax.set_ylabel('EMISSIONS (MT)')
ax.set_xlabel('')
plt.locator_params(axis='x', nbins=11)
plt.xticks(rotation = -45)

for label in ax.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False)

# GSA Results

## Read in GSA Results

### Objective cost

### Model period emissions

### Renewable energy generation share in last year

### Append and sort mu* values

## Plot Results 

### mu* Results

### Method of Morris for renewable energy generation share
Our GSA has 39 sensitivity parameters and 10 trajectories, resulting in (39+1)*10 model runs. Each trajectory group is shown below.   